In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-8sz1f7zg7oy0ui"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


#### Temasek Foundation Dataset

In [2]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node 

tf_meta_prompt, test_cases, tf_metric_map = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 
node = EvolNode(tf_meta_prompt, None, None, get_response=get_endpoint_response, test_cases=test_cases[:90],
                custom_metric_map=tf_metric_map) # setting manual test cases


In [14]:
node.evolve("i1", batch_size=5)

import time 
query_node = True 
replace = True 
feedback = ""
batch_size = 5
self = node 
method = "i1"
parents = None 

start_time = time.time()
# Query once
offsprings = [] 
if query_node:
    self.query_nodes(ignore_self=replace, self_func_name=self.meta_prompt.func_name)
query_end_time = time.time()
query_time = query_end_time - start_time
print(f"     :: Query time: {query_time:.2f}s")

# Evolve many times
reasonings, codes = self._evolve(method, parents, feedback=feedback, batch_size=batch_size)
evolve_end_time = time.time()
evolve_time = evolve_end_time - query_end_time
print(f"     :: Evolution time: {evolve_time:.2f}s")

self.reasonings = reasonings
self.codes = codes

     :: Query time: 23.86s


Processing LLM queries: 100%|██████████| 5/5 [01:01<00:00, 12.26s/it]

 :: Total time elapsed: 61.30s, 0 errors
     :: Evolution time: 61.33s


import json


def generate_prompt(project_description):
    prompt = f"""Evaluate the grant application for the project described below: 

{project_description}

"""
    prompt += (
        "Follow the output structure: {'label': str(...), 'comment': str(...)}} in a markdown format."
        )
    prompt += (
        ' Make sure to only provide the final decision and a brief comment explaining your decision.'
        )
    return prompt



In [13]:
input_dicts = test_cases[:90]

from tqdm import tqdm 
import types 
from methods.meta_execute import Timeout_
from collections import defaultdict

code = 'def generate_prompt(project_description):\n    decision = \'Maybe\'\n    comment = \'No reasonable decision can be made without further information.\'\n    prompt = (\n        f"Given the project description {{project_description= \'{project_description}\' }}, do evaluate the grant application, make a decision (Yes, No, Maybe) and a brief comment explaining why the project is likely to be accepted or rejected. Make sure the output is in JSON format in markdown like this {{\'label\': \'str()\', \'comment\': \'str()\'}} near the top of your response."\n        )\n    return prompt'
codes = [code]
max_tries = 3

prompts = []
input_indices = []
errors_per_code_per_test = defaultdict(lambda: defaultdict(list))

total_iterations = len(input_dicts) * len(codes)
with tqdm(total=total_iterations, desc="Executing Node to generate prompts ...") as pbar:
    for (input_index, input_dict) in enumerate(input_dicts):
        for (code_index, code) in enumerate(codes):
            mod = types.ModuleType('dynamic_module')
            try:
                with Timeout_(3):  # 3-second timeout
                    exec(code, mod.__dict__)
                    func_name = "generate_prompt"
                    assert func_name in mod.__dict__, f"Function {func_name} not found in code #{code_index}"
                    prompt_func = mod.__dict__[func_name]
                    prompt = prompt_func(**input_dict)
                    
                    for _ in range(max_tries): # number of trials
                        prompts.append(prompt)
                        input_indices.append((input_index, code_index))
                        
                errors_per_code_per_test[code_index][input_index] = [] # empty error when one trial works 
                        
            except Exception as e:
                errors_per_code_per_test[code_index][input_index].append(str(e))
            
            pbar.update(1)

Executing Node to generate prompts ...: 100%|██████████| 90/90 [00:00<00:00, 30291.07it/s]
